##### vix 변동률 구하기
* 전일 변동율 = (전날종가 - 전날시가) / 전날시가 -> vix_day_change
* 전일 변동폭 = 전날고가 - 전날저가 -> vix_day_fluc_range
* (종가 - 종가) 변동율 = (당일종가 - 전날종가) / 전날종가 -> vix_close_change -> 변동 %
* 데이터 기간 : 2023/07/31 ~ 2023/10/20

In [32]:
import pandas as pd

In [33]:
vix = pd.read_csv('../dataset/VIX.csv')
vix

,날짜,종가,시가,고가,저가,거래량,변동 %
0,2023- 10- 23,22.41,21.83,23.08,21.77,NaN,3.22%
1,2023- 10- 20,21.71,21.59,21.83,20.42,NaN,1.45%
2,2023- 10- 19,21.40,19.73,21.40,18.55,NaN,11.34%
3,2023- 10- 18,19.22,18.36,20.15,17.88,NaN,7.49%
4,2023- 10- 17,17.88,17.41,18.54,16.97,NaN,3.89%
...,...,...,...,...,...,...,...
103,2023- 05- 31,17.94,18.04,18.40,17.12,NaN,2.75%
104,2023- 05- 30,17.46,17.56,18.34,16.98,NaN,0.00%
105,2023- 05- 29,17.46,17.53,17.60,17.34,NaN,-2.73%
106,2023- 05- 26,17.95,19.07,19.56,17.27,NaN,-6.22%


In [34]:
vix.날짜 = pd.to_datetime(vix.날짜)

In [35]:
vix.drop(['거래량'], axis=1, inplace=True)

In [36]:
vix['변동 %'] = vix['변동 %'].astype(str)

In [37]:
vix["변동 %"] = vix["변동 %"].str.rstrip('%')
vix["변동 %"] = vix["변동 %"].astype(float)

In [38]:
vix.head(5)

,날짜,종가,시가,고가,저가,변동 %
0,2023-10-23,22.41,21.83,23.08,21.77,3.22
1,2023-10-20,21.71,21.59,21.83,20.42,1.45
2,2023-10-19,21.40,19.73,21.40,18.55,11.34
3,2023-10-18,19.22,18.36,20.15,17.88,7.49
4,2023-10-17,17.88,17.41,18.54,16.97,3.89


In [39]:
# close to close 변동률은 한국 기준 시간에서 
vix['변동 %'] = vix['변동 %'].shift(-1)

In [40]:
vix['vix_day_change'] = 0

In [43]:
vix.head(5)

,날짜,종가,시가,고가,저가,변동 %,vix_day_change
0,2023-10-23,22.41,21.83,23.08,21.77,1.45,0
1,2023-10-20,21.71,21.59,21.83,20.42,11.34,0
2,2023-10-19,21.40,19.73,21.40,18.55,7.49,0
3,2023-10-18,19.22,18.36,20.15,17.88,3.89,0
4,2023-10-17,17.88,17.41,18.54,16.97,-10.92,0


In [45]:
vix['vix_day_change'] = ((vix['종가'] - vix['시가']) / vix['시가']) * 100

In [47]:
vix['vix_day_fluc_range'] = vix['고가'] - vix['저가']

In [49]:
vix['vix_day_change'] = vix['vix_day_change'].shift(-1)
vix['vix_day_fluc_range'] = vix['vix_day_fluc_range'].shift(-1)

In [52]:
vix = vix[['날짜', '변동 %', 'vix_day_change', 'vix_day_fluc_range']]

In [56]:
vix = vix.rename(columns={'변동 %' : 'vix_close_change'})

##### 날짜 미루는거까지 완료 한국 날짜와 맞추기 위해 vix test 데이터로 관찰

In [61]:
vix[1:61].to_csv('../vix_test.csv')